
Challange-1

**Create Linear SVM Classifier without removing the correlated features and insignificant fetaures. Later apply those two feature engineering methods(removal of both correlated features and insignificant fetaures) see the differences.**

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [5]:
data = pd.read_csv("data.csv")

In [11]:
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1.0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1.0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1.0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1.0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1.0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
data.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [9]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.drop(data.columns[[-1]], axis=1, inplace=True)

In [10]:
data['diagnosis'] = data['diagnosis'].map({'M': 1.0, 'B': -1.0})

In [12]:
X = data.iloc[:, 1:]
Y = data.loc[:, 'diagnosis']

In [13]:
X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)

In [14]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

In [15]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier()

In [16]:
sklearn_pred = clf.predict(X_test)
print(accuracy_score(y_test, sklearn_pred))
print(recall_score(y_test, sklearn_pred))
print(recall_score(y_test, sklearn_pred))

0.9824561403508771
0.9534883720930233
0.9534883720930233


In [17]:
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train)
sklearn_svm = clf.predict(X_test)
print(accuracy_score(y_test, sklearn_svm))
print(recall_score(y_test, sklearn_svm))
print(recall_score(y_test, sklearn_svm))

0.9736842105263158
0.9534883720930233
0.9534883720930233


In [18]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped

In [19]:
def remove_less_significant_features(X, Y):
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped

In [24]:
data = pd.read_csv('./data.csv')
data.drop(data.columns[[-1, 0]], axis=1, inplace=True)
diag_map = {'M': 1.0, 'B': -1.0}
data['diagnosis'] = data['diagnosis'].map(diag_map)
Y = data.loc[:, 'diagnosis']
X = data.iloc[:, 1:]
remove_correlated_features(X)
remove_less_significant_features(X, Y)
X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)
X.insert(loc=len(X.columns), column='intercept', value=1)
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

In [25]:
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)

SGDClassifier()

In [26]:
sklearn_pred = clf.predict(X_test)

print("accuracy on test dataset: {}".format(accuracy_score(y_test, sklearn_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, sklearn_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, sklearn_pred)))

accuracy on test dataset: 0.8771929824561403
recall on test dataset: 1.0
precision on test dataset: 1.0


In [27]:
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train)
sklearn_svm = clf.predict(X_test)
sklearn_pred = clf.predict(X_test)
print("accuracy on test dataset: {}".format(accuracy_score(y_test, sklearn_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, sklearn_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, sklearn_pred)))

accuracy on test dataset: 0.9736842105263158
recall on test dataset: 0.9302325581395349
precision on test dataset: 0.9302325581395349
